# Seleção de Atributos (variáveis)

# Métricas

### Selecionar os melhores atributos, depois treinar o modelo e aplicar as métricas

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Criando um modelo
from sklearn import linear_model

def calcula_metricas(dataset, atributos_selecionados):

    X = dataset.loc[:, atributos_selecionados ].values
    y = dataset['target'].values

    modelo = linear_model.LinearRegression() 
    modelo.fit(X,y)

    # MAE - Mean Absolute Error
    from sklearn import model_selection
    kfold = 10
    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, 
                                                scoring = 'neg_mean_absolute_error')

    # Print do resultado
    print("MAE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

    # MSE - Mean Squared Error
    # Definindo os valores para o número de folds
    num_folds = 10
    num_instances = len(X)
    seed = 7

    # Separando os dados em folds
    kfold = model_selection.KFold(num_folds, True, random_state = seed)

    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'neg_mean_squared_error')

    # Print do resultado
    print("MSE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

    # RMSE (Root Mean Squared Error
    from math import sqrt
    print("RMSE: %.3f " % (sqrt(abs(resultado.mean()))))

    # R2
    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'r2')

    # Print do resultado
    print("R^2: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

# Seleciona os atributos
atributos_selecionados = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', # 'DIS', 
                              'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'] # 'AGE',

#calcula_metricas(dataset, atributos_selecionados)

## Carregando o Dataset Boston Houses

1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-residential acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per 10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. TARGET: Median value of owner-occupied homes in $1000's

## Seleção de Atributos - Utilizando RFE - Recursive Feature Elimination

Esta técnica para seleção de atributos, que recursivamente remove os atributos e constrói o modelo com os atributos remanescentes. <br> Esta técnica utiliza a acurácia do modelo para identificar os atributos que mais contribuem para prever a variável alvo. <br> Em inglês esta técnica é chamada Recursive Feature Elimination (RFE).


In [2]:
# Gerando o dataset
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target
dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
print('Número de Atributos:', len(dataset.columns) -1)

Número de Atributos: 13


In [4]:
dataset.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'target'],
      dtype='object')

# Aplicando a RFE para Seleção dos Melhores Atributos

In [6]:
%%time 

# aplicando a RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


X = dataset.iloc[:,:-1].values
y = dataset['target'].values

num_atributos_relevantes = 8
estimator = SVR(kernel="linear")
selector = RFE(estimator, num_atributos_relevantes, step=1)
selector = selector.fit(X, y)

print("Num Features: ", selector.n_features_)

print(dataset.columns)
print("Selected Features: ", selector.support_)
print("Feature Ranking: ", selector.ranking_)

Num Features:  8
Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'target'],
      dtype='object')
Selected Features:  [ True False  True  True  True  True False  True False False  True False
  True]
Feature Ranking:  [1 2 1 1 1 1 3 1 5 6 1 4 1]
Wall time: 12.7 s


In [7]:
dfatributos = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': selector.ranking_ })
                             
dfatributos = dfatributos.sort_values(by='Importancia', ascending=True)
dfatributos

,Atributo,Importancia
0,CRIM,1
2,INDUS,1
3,CHAS,1
4,NOX,1
5,RM,1
7,DIS,1
10,PTRATIO,1
12,LSTAT,1
1,ZN,2
6,AGE,3


In [8]:
dfatributos.Atributo.values

array(['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT',
       'ZN', 'AGE', 'B', 'RAD', 'TAX'], dtype=object)

# Comparação de Modelos

In [11]:
print("Modelo com 13 atributos")
atributos_selecionados = ['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT',
       'ZN', 'AGE', 'B', 'RAD', 'TAX']
calcula_metricas(dataset, atributos_selecionados)

print("\nModelo com atributos selecionados com RFE")
# Seleciona os atributos
atributos_selecionados = ['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']
       #'ZN', 'AGE', 'B', 'RAD', 'TAX']

calcula_metricas(dataset, atributos_selecionados)

Modelo com 13 atributos
MAE: -4.005 (2.084)
MSE: -23.747 (11.143)
RMSE: 4.873 
R^2: 0.718 (0.099)

Modelo com atributos selecionados com RFE
MAE: -3.872 (1.752)
MSE: -24.967 (11.602)
RMSE: 4.997 
R^2: 0.701 (0.104)


## Seleção de Atributos -  Seleciona os atributos pela variância

In [12]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
selector.fit_transform(X)
selector.variances_

array([7.38403597e+01, 1.00000000e+02, 2.72800000e+01, 6.43854770e-02,
       1.34010989e-02, 4.92695216e-01, 9.71000000e+01, 4.42525226e+00,
       2.30000000e+01, 5.24000000e+02, 4.67772630e+00, 3.96580000e+02,
       3.62400000e+01])

In [13]:
dfatributos2 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': selector.variances_  })
                             
dfatributos2 = dfatributos2.sort_values(by='Importancia', ascending=True).reset_index(drop=True)
dfatributos2

,Atributo,Importancia
0,NOX,0.013401
1,CHAS,0.064385
2,RM,0.492695
3,DIS,4.425252
4,PTRATIO,4.677726
5,RAD,23.000000
6,INDUS,27.280000
7,LSTAT,36.240000
8,CRIM,73.840360
9,AGE,97.100000


In [14]:
dfatributos2.Atributo.values

array(['NOX', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'RAD', 'INDUS', 'LSTAT',
       'CRIM', 'AGE', 'ZN', 'B', 'TAX'], dtype=object)

In [16]:
print("Modelo com 13 atributos")
atributos_selecionados = ['NOX', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'RAD', 'INDUS', 'LSTAT',
       'CRIM', 'AGE', 'ZN', 'B', 'TAX']
calcula_metricas(dataset, atributos_selecionados)

print("\nModelo com atributos selecionados com VarianceThreshold")
# Seleciona os atributos
atributos_selecionados = ['NOX', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'RAD', 'INDUS', 'LSTAT',
       'CRIM', 'AGE'] #'ZN', 'B', 'TAX']

calcula_metricas(dataset, atributos_selecionados)

Modelo com 13 atributos
MAE: -4.005 (2.084)
MSE: -23.747 (11.143)
RMSE: 4.873 
R^2: 0.718 (0.099)

Modelo com atributos selecionados com VarianceThreshold
MAE: -4.126 (1.883)
MSE: -24.843 (11.326)
RMSE: 4.984 
R^2: 0.704 (0.101)


## Seleção de Atributos - Utilizando Árvores de Decisão 

Bagged Decision Trees, como o algoritmo RandomForest, podem ser usados para estimar a importância de cada atributo. Esse método retorna um score para cada atributo.

Quanto maior o score, maior a importância do atributo.

In [17]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(n_estimators=30, max_depth=6)
clf = clf.fit(X, y)
clf.feature_importances_           

array([3.97291687e-02, 1.99950440e-04, 3.68880719e-03, 5.53901300e-04,
       1.69614865e-02, 4.64445989e-01, 6.37329271e-03, 7.52483626e-02,
       1.67601031e-03, 1.30807071e-02, 1.21339173e-02, 9.89669481e-03,
       3.56011712e-01])

In [18]:
dfatributos3 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': clf.feature_importances_  })
                             
dfatributos3 = dfatributos3.sort_values(by='Importancia', ascending=False)
dfatributos3

,Atributo,Importancia
5,RM,0.464446
12,LSTAT,0.356012
7,DIS,0.075248
0,CRIM,0.039729
4,NOX,0.016961
9,TAX,0.013081
10,PTRATIO,0.012134
11,B,0.009897
6,AGE,0.006373
2,INDUS,0.003689


# Atividade 1-A: Criar o modelo e as métricas, comparando com o modelo original

## Seleção de Atributos - Extra Trees Classifier

In [21]:
# Feature Importance with Extra Trees Classifier

from sklearn.ensemble import ExtraTreesRegressor

# feature extraction
model = ExtraTreesRegressor()
model.fit(X, y)
print(model.feature_importances_)

[0.03045685 0.00427296 0.04744668 0.01571341 0.05721691 0.33242963
 0.01816634 0.03132951 0.01874066 0.03815305 0.04324354 0.01822768
 0.34460276]


In [22]:
dfatributos4 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': model.feature_importances_ })
                             
dfatributos4 = dfatributos3.sort_values(by='Importancia', ascending=False).reset_index(drop=True)
dfatributos4

,Atributo,Importancia
0,RM,0.464446
1,LSTAT,0.356012
2,DIS,0.075248
3,CRIM,0.039729
4,NOX,0.016961
5,TAX,0.013081
6,PTRATIO,0.012134
7,B,0.009897
8,AGE,0.006373
9,INDUS,0.003689


In [23]:
# Seleciona os atributos
atributos_selecionados = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', # 'DIS', 
                              'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'] # 'AGE',

calcula_metricas(dataset, atributos_selecionados)

MAE: -4.149 (2.069)
MSE: -26.130 (12.988)
RMSE: 5.112 
R^2: 0.692 (0.112)


# Atividade 1-B: Criar o modelo e as métricas, comparando com o modelo original